In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/my projects/breast_cancer_detection/data/data.csv")

In [ ]:
features=list(data.keys())

In [ ]:
features.remove("Unnamed: 32")

In [ ]:

print(features[0])

id


In [ ]:
id = list(data[features[0]])
Y_list = list(data[features[1]])

In [ ]:
def one_hot_for_y(Y_list):
  m = len(Y_list)
  Y = np.zeros((m,1))
  for count,diagnosis in enumerate(Y_list):
    if(diagnosis == "M"):
      Y[count,0] = 1
  return Y


In [ ]:
Y=one_hot_for_y(Y_list)

In [ ]:
features.remove("diagnosis")
features.remove("id")

In [ ]:
def build_X(features,data):
  n = len(features)
  m = len(data[features[0]])
  X = np.zeros((m,n))
  for count,feature in enumerate(features):
    X[:,count] = np.array(list(data[feature]))
  return X

In [ ]:
X = build_X(features,data)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

In [ ]:
print(X_train.shape)

(362, 30)


In [ ]:
import keras
from keras.layers import Input,Dense,Activation,Dropout,BatchNormalization
from keras.initializers import glorot_uniform
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.regularizers import l2


In [ ]:
def cancer_model(input_shape):
  X_input = Input(shape=input_shape,dtype='float64')
  X = Dense(60,activation='relu',kernel_initializer=glorot_uniform(),kernel_regularizer=l2(0.01))(X_input)
  X = BatchNormalization()(X)
  X = Dropout(rate=0.2)(X)
  X = Dense(60,activation='relu',kernel_initializer=glorot_uniform(),kernel_regularizer=l2(0.01))(X)
  X = BatchNormalization()(X)
  X = Dropout(rate=0.2)(X)
  X = Dense(40,activation='relu',kernel_initializer=glorot_uniform(),kernel_regularizer=l2(0.01))(X)
  X = Dense(1,activation='sigmoid',kernel_initializer=glorot_uniform())(X)
  
  model = Model(inputs=X_input,outputs=X)
  return model

In [ ]:
model = cancer_model((X.shape[1],))

In [ ]:
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
dense_16 (Dense)             (None, 60)                1860      
_________________________________________________________________
batch_normalization_8 (Batch (None, 60)                240       
_________________________________________________________________
dropout_8 (Dropout)          (None, 60)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 60)                3660      
_________________________________________________________________
batch_normalization_9 (Batch (None, 60)                240       
_________________________________________________________________
dropout_9 (Dropout)          (None, 60)                0   

In [ ]:
opt = Adam(learning_rate=0.00001)
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=218)
# model.fit(X,Y,epochs=16)

Epoch 1/218
12/12 [==============================] - 1s 20ms/step - loss: 0.1362 - accuracy: 0.9513 - val_loss: 0.0975 - val_accuracy: 0.9835
Epoch 2/218
12/12 [==============================] - 0s 6ms/step - loss: 0.1873 - accuracy: 0.9474 - val_loss: 0.0977 - val_accuracy: 0.9835
Epoch 3/218
12/12 [==============================] - 0s 6ms/step - loss: 0.1287 - accuracy: 0.9637 - val_loss: 0.0976 - val_accuracy: 0.9835
Epoch 4/218
12/12 [==============================] - 0s 6ms/step - loss: 0.1419 - accuracy: 0.9619 - val_loss: 0.0974 - val_accuracy: 0.9835
Epoch 5/218
12/12 [==============================] - 0s 6ms/step - loss: 0.2217 - accuracy: 0.9084 - val_loss: 0.0977 - val_accuracy: 0.9835
Epoch 6/218
12/12 [==============================] - 0s 6ms/step - loss: 0.1217 - accuracy: 0.9614 - val_loss: 0.0977 - val_accuracy: 0.9835
Epoch 7/218
12/12 [==============================] - 0s 6ms/step - loss: 0.1262 - accuracy: 0.9449 - val_loss: 0.0975 - val_accuracy: 0.9835
Epoch 8/218


In [ ]:
test_loss,test_acc=model.evaluate(X_test,y_test,verbose=1)
print(test_acc)

3/3 [==============================] - 0s 3ms/step - loss: 0.1732 - accuracy: 0.9535
0.9534883499145508


In [ ]:
def get_probability(X):
  prediction =( model.predict(X) ) * 100
  pred_str = "Probability that you have breast cancer(malignant) is "+str(prediction) + " %"
  return pred_str


In [ ]:
my_input = X_test[5,:].reshape(1,30)
print(get_probability(my_input))

Probability that you have breast cancer(malignant) is [[98.9833]] %


In [ ]:
model.save('/content/drive/MyDrive/my projects/breast_cancer_detection/model/breast_cancer.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/my projects/breast_cancer_detection/model/breast_cancer.h5')

                    **Data from other site testing**

---



In [ ]:
df = pd.read_csv("/content/drive/MyDrive/my projects/breast_cancer_detection/other_site_data.csv")

In [ ]:
df.head()

,842302,M,17.99,10.38,122.8,1001,0.1184,0.2776,0.3001,0.1471,0.2419,0.07871,1.095,0.9053,8.589,153.4,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.6,2019,0.1622,0.6656,0.7119,0.2654,0.4601,0.1189
0,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
1,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
2,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
3,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678
4,843786,M,12.45,15.70,82.57,477.1,0.12780,0.17000,0.1578,0.08089,0.2087,0.07613,0.3345,0.8902,2.217,27.19,0.007510,0.03345,0.03672,0.01137,0.02165,0.005082,15.47,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440


In [ ]:
arr = np.array(df)

In [ ]:
print(arr.shape)

(568, 32)


In [ ]:
y = arr[:,1]

In [ ]:
print(y)

['M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'B' 'B' 'B' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'B' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'B' 'M' 'B' 'B' 'B' 'B' 'B' 'M' 'M'
 'B' 'M' 'M' 'B' 'B' 'B' 'B' 'M' 'B' 'M' 'M' 'B' 'B' 'B' 'B' 'M' 'B' 'M'
 'M' 'B' 'M' 'B' 'M' 'M' 'B' 'B' 'B' 'M' 'M' 'B' 'M' 'M' 'M' 'B' 'B' 'B'
 'M' 'B' 'B' 'M' 'M' 'B' 'B' 'B' 'M' 'M' 'B' 'B' 'B' 'B' 'M' 'B' 'B' 'M'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'M' 'M' 'M' 'B' 'M' 'M' 'B' 'B' 'B' 'M'
 'M' 'B' 'M' 'B' 'M' 'M' 'B' 'M' 'M' 'B' 'B' 'M' 'B' 'B' 'M' 'B' 'B' 'B'
 'B' 'M' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'M' 'B' 'B' 'B' 'B' 'M' 'M'
 'B' 'M' 'B' 'B' 'M' 'M' 'B' 'B' 'M' 'M' 'B' 'B' 'B' 'B' 'M' 'B' 'B' 'M'
 'M' 'M' 'B' 'M' 'B' 'M' 'B' 'B' 'B' 'M' 'B' 'B' 'M' 'M' 'B' 'M' 'M' 'M'
 'M' 'B' 'M' 'M' 'M' 'B' 'M' 'B' 'M' 'B' 'B' 'M' 'B' 'M' 'M' 'M' 'M' 'B'
 'B' 'M' 'M' 'B' 'B' 'B' 'M' 'B' 'B' 'B' 'B' 'B' 'M' 'M' 'B' 'B' 'M' 'B'
 'B' 'M' 'M' 'B' 'M' 'B' 'B' 'B' 'B' 'M' 'B' 'B' 'B

In [ ]:
x = arr[:,2:]

In [ ]:
print(x.dtype)

object


In [ ]:
y = one_hot_for_y(list(y))

In [ ]:
print(y.shape)

(568, 1)


In [ ]:
x = x.astype(np.float64)

In [ ]:
my_input = x[19,:].reshape(1,30)

In [ ]:
print(my_input.dtype)

float64


In [ ]:
print(model.predict(my_input),y[19,0])

[[0.00468742]] 0.0


In [ ]:
test_loss,test_acc=model.evaluate(x,y)
print(test_acc)

18/18 [==============================] - 0s 2ms/step - loss: 0.1169 - accuracy: 0.9683
0.9683098793029785


In [ ]:
print(f"Other site data is giving accuracy of {test_acc * 100} %")

Other site data is giving accuracy of 96.83098793029785 %
